# One Node Simulation

## Importing packages:

In [20]:
import sys
import os
import netsquid as ns
import pydynaa
import pandas
import numpy as np

from numpy import random
from itertools import combinations

from netsquid.nodes import Node, Connection, Network

from netsquid.protocols import Protocol, NodeProtocol, Signals

from netsquid.util.datacollector import DataCollector

import netsquid.qubits.ketstates as ks

from netsquid.examples.teleportation import example_network_setup
from netsquid.qubits import qubitapi as qapi
from netsquid.qubits.qubitapi import ops
from netsquid.qubits import StateSampler

from netsquid.components import ClassicalChannel, QuantumChannel, QuantumProgram, QuantumProcessor, INSTR_MEASURE
from netsquid.components.qsource import QSource, SourceStatus
from netsquid.components.models import DelayModel, FixedDelayModel, FibreDelayModel, FibreLossModel, DephaseNoiseModel, DepolarNoiseModel
from netsquid.components.qprocessor import QuantumProcessor, PhysicalInstruction
from netsquid.components.instructions import INSTR_ROT_Z, INSTR_INIT, INSTR_EMIT, IMeasureFaulty

import netsquid.components.instructions as instr

import import_ipynb
from NoiseModels import EmissionNoiseModel, CollectiveDephasingNoiseModel
from Instructions import IonTrapMultiQubitRotation, IonTrapMSGate, INSTR_INIT_BELL
from nstrappedions.InitPhoton import SendPhoton
from programs import emit_prog

importing Jupyter notebook from programs.ipynb


#### testing import successful

In [4]:
tester1 = IonTrapMSGate(2)
tester1

Instruction: ms_gate_theta=0.5_pi

In [5]:
tester2 = SendPhoton(2)
tester2

SendPhoton(name='ion_trap_quantum_communication_device')

In [6]:
#emission_validation()

## 1. Initialize

In [8]:
# initalizing ba_ion + photon

def setup(collection_efficiency, fiber_length, attenutation):
    #attenuation in dB/km
    #from thor labs attenuation = 30
    #fiber_length = lenght before beam splitter + length after beam splitter = 2 + 2 = 4 m
    #this setup avoids accounting for noise from beamsplitter
    #https://www.thorlabs.com/newgrouppage9.cfm?objectgroup_id=1362
    #https://www.thorlabs.com/drawings/393bc612d06a8b2b-22BB6E3C-CFFC-5230-EB01198D395F1B14/SM450-SpecSheet.pdf
    
    ion_trap = SendPhoton(num_postions=1, collection_efficiency=collection_efficiency)
    # set collection_efficiency = (1 - 0.1%) when declaring
    # this shold account for S_1/2 to P_1/2 Ba transition
    # collection_efficiency =  probability that the qubit is not lost during emission. 
    
    fiber_loss_model = FibreLossModel(p_loss_length=attenuation, p_loss_init=.1e-2)
    #setting p_loss_init = .01 (99% usually Ba excitation works)
    #double counting for phenomenon in iontrap thru colleciton_efficiency and here thru p_loss_init
    fiber_delay_model = FibreDelayModel(c=3e8/1.5)
    
    fiber = QuantumChannel(name="fiber", length=fiber_length,
                          models={"quantum_loss_model": fiber_loss_model, 
                                 "delay_model": fiber_delay_model},
                          transmit_empty_items=True)
    
    fiber.ports["send"].connect(ion_trap.ports["qout"])
    collection_port = fiber.ports["recv"]

    # collection_port = ion_trap.ports["qout"]

    return ion_trap, collection_port


In [9]:
#if executing idea2 in measure, try declaring qubits something like this:


#q1, q2 = ns.qubits.create_qubits(2)  # separate states |0>, |0>

# 2. Emission

In [10]:
class EmitProg(QuantumProgram):

    default_num_qubits = 2

    def program(self):
        memory_position, emission_position = self.get_qubit_indices()
        self.apply(instruction=INSTR_INIT, qubit_indices=memory_position)
        self.apply(instruction=INSTR_EMIT, qubit_indices=[memory_position, emission_position])
        yield self.run()

In [11]:
emit_prog = EmitProg()
emit_prog

In [12]:
# figure out how to handle no photon detected at phtoonic entangler
# code below from emission_validation (implementing idea2: ad probability portal proceeds either way, must check lke emission_validation protocol if geting expected proceed prob)

"""

# ns.logger.setLevel(logging.DEBUG)

# x = np.arange(0.1, 1, 0.1)
# xstring = "collection efficiency"

x = range(1, 10)
xstring = "fiber length"

collection_efficiency = 1
fiber_length = 1
attenuation = 0.25

success_probs = []
success_prob_errors = []
expected_success_probs = []

for fiber_length in x:

    ion_trap, collection_port = setup(collection_efficiency=collection_efficiency,
                                      fiber_length=fiber_length,
                                      attenuation=attenuation)
    fail_count = 0
    num_tries = 500
    outcomes = []
    for _ in range(num_tries):
        ion_trap.execute_program(emit_prog)
        ns.sim_run()
        emitted_message = collection_port.rx_output()
        emitted_qubit = emitted_message.items[0]
        if emitted_qubit is None:
            outcomes.append(0)
        else:
            if emitted_qubit.qstate is None:
                outcomes.append(0)
            else:
                outcomes.append(1)

    success_prob = np.mean(outcomes)
    success_probs.append(success_prob)
    success_prob_error = np.std(outcomes) / math.sqrt(len(outcomes))
    success_prob_errors.append(success_prob_error)
    expected_success_prob = collection_efficiency * np.power(10, - attenuation * fiber_length / 10)
    expected_success_probs.append(expected_success_prob)
"""

"""
plt.figure()
plt.errorbar(x, y=success_probs, yerr=success_prob_errors, label="measured")
plt.plot(x, expected_success_probs, label="expected")
plt.xlabel(xstring)
plt.ylabel("success probability)")
plt.legend()
plt.show()
"""

'\nplt.figure()\nplt.errorbar(x, y=success_probs, yerr=success_prob_errors, label="measured")\nplt.plot(x, expected_success_probs, label="expected")\nplt.xlabel(xstring)\nplt.ylabel("success probability)")\nplt.legend()\nplt.show()\n'

## 3. Measure

In [13]:
# idea2: 
#observable (Operator, optional) – Hermitian operator to measure qubit with. Default is the Z observable.


#combine_qubits([q1, q2])  # state |00>
ns.qubits.measure(q2) 

(0, 1.0)

In [14]:
print(ns.qubits.measure(q2))
print(ns.qubits.measure(q1))
print(ns.qubits.reduced_dm(q2))  #how to get reduced density matrix
print(ns.qubits.reduced_dm(q1))  #how to get reduced density matrix

(0, 1.0)
(0, 1.0)
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


In [15]:
# idea1: 

def ion_trap_meas_prog(meas_basis):

    if meas_basis != "X" and meas_basis != "Z":
        raise ValueError("Measurement basis should be either X or Z")
    prog = QuantumProgram(num_qubits=1, parallel=False)
    q = prog.get_qubit_indices()
    if meas_basis == "X":
        prog.apply(instruction=INSTR_ROT_Z, qubit_indices=q, angle=np.pi)
        prog.apply(IonTrapMultiQubitRotation(num_positions=1), qubit_indices=q, phi=np.pi / 2,
                   theta=np.pi / 2)
    prog.apply(INSTR_MEASURE, qubit_indices=q, output_key="outcome")

    return prog

In [16]:
# test idea1
ion_trap_meas_z = ion_trap_meas_prog("Z")
ion_trap_meas_x = ion_trap_meas_prog("X")
print(ion_trap_meas_z)
print(ion_trap_meas_x)

## 4. If Detected?

In [7]:
# initializng yb (movign this to ahndle after photonic entangnler)

class InitStateProgram(QuantumProgram):
    """Program to create a qubit and transform it to the y0 state.

    """
    default_num_qubits = 1

    def program(self):
        q1, = self.get_qubit_indices(1)
        self.apply(instr.INSTR_INIT, q1)
        self.apply(instr.INSTR_H, q1)
        self.apply(instr.INSTR_S, q1)
        yield self.run()

In [17]:
# if detected, proceed
# otherwise, resample()
# pull from nstrappedions

#example simulation setup with data collector for fidelities

def example_sim_setup(nodeA, nodeB):
    """Example ismulation setup with data collector for teleporation protocol.
    
    Parameters
    ----------
    node_A : :class:`~netsquid.nodes.node.Node`
        Node corresponding to Alice.
    node_B : :class:`~netsquid.nodes.node.Node`
        Node corresponding to Bob.

    Returns
    -------
    :class:`~netsquid.protocols.protocol.Protocol`
        Alice's protocol.
    :class:`~netsquid.protocols.protocol.Protocol`
        Bob's protocol.
    :class:`~netsquid.util.datacollector.DataCollector`
        Data collector to record fidelity.

    """
    
    def collect_fidelity_data(evexpr):
        protocol = evexpr.triggered_events[-1].source
        mem_pos = protocol.get_signal_result(Signals.SUCCESS)
        qubit, = protocol.node.qmemory.pop(mem_pos)
        fidelity = qapi.fidelity(qubit, ns.y0, squared = True)
        qapi.discard(qubit)
        return {"fidelity": fidelity}
    
    protocol_alice = BellMeasurementProtocol(nodeA)
    protocol_bob = CorrectionProtocol(nodeB)
    dc = DataCollector(collect_fidelity_data)
    dc.collect_on(pydynaa.EventExpression(source=protocol_bob, 
                                         event_type=Signals.SUCCESS.value))
    return protocol_alice, protocol_bob, dc

#run experiment

#measure average fidelity of 1000 runs and run that experiment for different depolarization rates
def run_experiment(num_runs, depolar_rates, distance=4e-3, dephase_rate=0.0):
    """Setup and run the simulation experiment.

    Parameters
    ----------
    num_runs : int
        Number of cycles to run teleportation for.
    depolar_rates : list of float
        List of depolarization rates to repeat experiment for.
    distance : float, optional
        Distance between nodes [km].
    dephase_rate : float, optional
        Dephasing rate of physical measurement instruction.

    Returns
    -------
    :class:`pandas.DataFrame`
        Dataframe with recorded fidelity data.

    """
    fidelity_data = pandas.DataFrame()
    for i, depolar_rate in enumerate(depolar_rates):
        ns.sim_reset()
        network = example_network_setup(distance, depolar_rate, dephase_rate)
        node_a = network.get_node("Alice")
        node_b = network.get_node("Bob")
        protocol_alice, protocol_bob, dc = example_sim_setup(node_a, node_b)
        protocol_alice.start()
        protocol_bob.start()
        q_conn = network.get_connection(node_a, node_b, label="quantum")
        cycle_runtime = (q_conn.subcomponents["qsource"].subcomponents["internal_clock"]
                         .models["timing_model"].delay)
        ns.sim_run(cycle_runtime * num_runs + 1)
        df = dc.dataframe
        df['depolar_rate'] = depolar_rate
        fidelity_data = fidelity_data.append(df)
    return fidelity_data

## 7. Output (get density matrix or get fidelity v. dephase graph)

In [18]:
# show figure when running full simulation

def create_plot():
    """Show a plot of fidelity verus depolarization rate.

    """
    from matplotlib import pyplot as plt
    dephase_rates = np.array([1e6 * i for i in range(0, 200, 10)])
    #coherence_times = 1/dephase_rates
    fidelities = run_experiment(num_runs=1000, distance=4e-3,
                                depolar_rates=0, dephase_rate=dephase_rates)
    plot_style = {'kind': 'scatter', 'grid': True,
                  'title': "Fidelity of the teleported quantum state"}
    data = fidelities.groupby("depolar_rate")['fidelity'].agg(
        fidelity='mean', sem='sem').reset_index()
    data.plot(x='depolar_rate', y='fidelity', yerr='sem', **plot_style)
    plt.show()